In [1]:
import sys
from glob import glob
from collections import defaultdict

sys.path.append("/scratch/group/csce435-f23/python-3.8.17/lib/python3.8/site-packages")
sys.path.append("/scratch/group/csce435-f23/thicket")

import pandas as pd

import thicket as th

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)

# POINT THIS AT YOUR DATA

In [3]:
# Point this at the top directory of all your cali files for all of the implementations. Other files can be in the directory too, that is ok.
FILES_LOCATION = "/scratch/user/anjalik02/Project-Bitonic"

# Reader check
Can the files be read in one-by-one

In [4]:
working_files = []
error_files = []
i = 0
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        tk = th.Thicket.from_caliperreader(file)
        working_files.append(file)
    except Exception:
        i += 1
        error_files.append(file)

In [5]:
print("Files that could be read in individually (one-by-one):")
print(f"{len(working_files)}/{len(team_files)} ({len(working_files)/len(team_files)*100}%)")

Files that could be read in individually (one-by-one):
264/264 (100.0%)


# Check for Metadata columns

Check for the necessary metadata columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3b-collect-metadata)

In [6]:
team_metadata_valid = []
metadata_columns=['cali.caliper.version', 'spot.options', 'spot.channels', 'cali.channel',
    'launchdate',
    'libraries', 'cmdline', 'cluster', 'Algorithm', 'ProgrammingModel',
    'Datatype', 'SizeOfDatatype', 'InputSize', 'InputType',
    'group_num', 'implementation_source']
mpi_cols = ['num_procs']
cuda_cols = ['num_threads', 'num_blocks',]
metadata_col_dict = defaultdict(lambda: [])

team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    try:
        valid = True
        tk = th.Thicket.from_caliperreader(file)
        cols = tk.metadata.columns
        model_to_check = []
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            model_to_check = metadata_columns + cuda_cols
        else:
            model_to_check = metadata_columns + mpi_cols
        for col in model_to_check:
            if col not in cols:
                metadata_col_dict[list(tk.profile_mapping.values())[0]].append(col)
                valid=False
        if valid:
            team_metadata_valid.append(file)
    except KeyError:
        pass

for file, cols in metadata_col_dict.items():
    print(f"File '{file}' missing metadata columns:\n\t{cols}")

# Check for DataFrame columns

Check for the necessary DataFrame columns from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#4c-you-should-measure-the-following-performance-metrics). For the GPU columns, you need one or the other column in the tuple, not both.

In [7]:
team_dataframe_valid = []
necessary_columns = ["Min time/rank","Max time/rank","Avg time/rank","Total time",]
not_gpu_columns = ["Variance time/rank",]
gpu_columns=[("Avg GPU time/rank", "Avg GPU Time/rank"),
             ("Min GPU time/rank", "Min GPU Time/rank"),
             ("Max GPU time/rank", "Max GPU Time/rank"),
             ("Total GPU time", "Total GPU Time"),]
def check_df_cols(tk, dict):
    valid = True
    cols = tk.dataframe.columns
    for col in necessary_columns:
        if col not in cols:
            dict[list(tk.profile_mapping.values())[0]].append(col)
            valid = False
    if "ProgrammingModel" in tk.metadata.columns:
        if "CUDA" in tk.metadata["ProgrammingModel"].to_list()[0].upper():
            for col in gpu_columns:
                if col[0] not in cols and col[1] not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
        else:
            for col in not_gpu_columns:
                if col not in cols:
                    dict[list(tk.profile_mapping.values())[0]].append(col)
                    valid = False
    return valid

dataframe_col_dict = defaultdict(lambda: [])
team_files = glob(f"{FILES_LOCATION}/**/*.cali", recursive=True)
for file in team_files:
    tk = th.Thicket.from_caliperreader(file)
    valid = check_df_cols(tk, dataframe_col_dict)
    if valid:
        team_dataframe_valid.append(file)

for file, cols in dataframe_col_dict.items():
    print(f"File '{file}' missing dataframe columns: {cols}")

# Try all files together

In [8]:
tk = th.Thicket.from_caliperreader(team_files)

# Check tree

Should be no different from the [report](https://github.com/TAMU-CSCE435-Pearce/Project/blob/master/Report.md#3a-caliper-instrumentation), spelling and all.

In [9]:
tk.statsframe.dataframe["time"] = 1
print(tk.tree())

  _____ _     _      _        _   
 |_   _| |__ (_) ___| | _____| |_ 
   | | | '_ \| |/ __| |/ / _ \ __|
   | | | | | | | (__|   <  __/ |_ 
   |_| |_| |_|_|\___|_|\_\___|\__|  v2023.3.0

1.000 main
├─ 1.000 comm
│  └─ 1.000 comm_large
│     ├─ 1.000 cudaMemcpy
│     ├─ 1.000 mpi_gather
│     └─ 1.000 mpi_scatter
├─ 1.000 comp
│  └─ 1.000 comp_large
├─ 1.000 correctness_check
└─ 1.000 data_init

Legend (Metric: time Min: 1.00 Max: 1.00)
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00
█ 1.00 - 1.00

name User code    ◀  Only in left graph    ▶  Only in right graph



In [10]:
# Groupby programming model. Should result in 2 thickets, MPI and CUDA.
gb_pmodel = tk.groupby("ProgrammingModel")

2  thickets created...
{'CUDA': <thicket.thicket.Thicket object at 0x2b7e4550cd30>, 'MPI': <thicket.thicket.Thicket object at 0x2b7e45449e80>}


In [11]:
# Groupby the parameters we ran with. After this operation, each Thicket in gb_total should contain profiles with unique InputSizes (there should be no duplicate input sizes).
gb_cuda = gb_pmodel["CUDA"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_threads"])
gb_mpi = gb_pmodel["MPI"].groupby(["ProgrammingModel", "Algorithm", "InputType", "num_procs"])
gb_total = {**gb_cuda, **gb_mpi}

20  thickets created...
{('CUDA', 'Bitonic Sort', '1% Perturbed', 64.0): <thicket.thicket.Thicket object at 0x2b7e45871d00>, ('CUDA', 'Bitonic Sort', '1% Perturbed', 128.0): <thicket.thicket.Thicket object at 0x2b7e458f5400>, ('CUDA', 'Bitonic Sort', '1% Perturbed', 256.0): <thicket.thicket.Thicket object at 0x2b7e457f6fd0>, ('CUDA', 'Bitonic Sort', '1% Perturbed', 512.0): <thicket.thicket.Thicket object at 0x2b7e455733d0>, ('CUDA', 'Bitonic Sort', '1% Perturbed', 1024.0): <thicket.thicket.Thicket object at 0x2b7e458ed160>, ('CUDA', 'Bitonic Sort', 'Randomized', 64.0): <thicket.thicket.Thicket object at 0x2b7e455e6b20>, ('CUDA', 'Bitonic Sort', 'Randomized', 128.0): <thicket.thicket.Thicket object at 0x2b7e45601c40>, ('CUDA', 'Bitonic Sort', 'Randomized', 256.0): <thicket.thicket.Thicket object at 0x2b7e4589bf10>, ('CUDA', 'Bitonic Sort', 'Randomized', 512.0): <thicket.thicket.Thicket object at 0x2b7e45583910>, ('CUDA', 'Bitonic Sort', 'Randomized', 1024.0): <thicket.thicket.Thicket ob

In [12]:
# Compose all of the data back together. If this step errors, you probably have duplicate inputsizes. Run 1a to check for this.
ctk = th.Thicket.concat_thickets(
    thickets=list(gb_total.values()),
    axis="columns",
    headers=list(gb_total.keys()),
    metadata_key="InputSize"
)

/scratch/group/csce435-f23/thicket/thicket/ensemble.py:248: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  combined_th.dataframe.loc[node, "name"] = node.frame["name"]


In [13]:
ctk.dataframe.head(50)

(CUDA, Bitonic Sort, 1% Perturbed, 64.0)  \
                                                                                           nid   
node                                        InputSize                                            
{'name': 'main', 'type': 'function'}        65536                                            1   
                                            262144                                           1   
                                            1048576                                          1   
                                            4194304                                          1   
                                            16777216                                         1   
                                            67108864                                         1   
                                            268435456                                        1   
{'name': 'comm', 'type': 'function'}        65536                                            3   
                                            262144                                           3   
                                            1048576                                          3   
                                            4194304                                          3   
                                            16777216                                         3   
                                            67108864                                         3   
                                            268435456                                        3   
{'name': 'comm_large', 'type': 'function'}  65536                                            4   
                                            262144                                           4   
                                            1048576                                          4   
                                            4194304                                          4   
                                            16777216                                         4   
                                            67108864                                         4   
                                            268435456                                        4   
{'name': 'cudaMemcpy', 'type': 'function'}  65536                                            5   
                                            262144                                           5   
                                            1048576                                          5   
                                            4194304                                          5   
                                            16777216                                         5   
                                            67108864                                         5   
                                            268435456                                        5   
{'name': 'mpi_gather', 'type': 'function'}  65536                                         <NA>   
                                            262144                                        <NA>   
                                            1048576                                       <NA>   
                                            4194304                                       <NA>   
                                            16777216                                      <NA>   
                                            67108864                                      <NA>   
                                            268435456                                     <NA>   
{'name': 'mpi_scatter', 'type': 'function'} 65536                                         <NA>   
                                            262144                                        <NA>   
                                            1048576                                       <NA>   
                                    

# 1A

Check for duplicate input sizes

In [27]:
i = 0
for key in list(gb_total.keys()):
    print(i)
    print(gb_total[key].profile_mapping)
    print(gb_total[key].metadata["InputSize"])
    i += 1

0
OrderedDict([(486381914, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t64-s262144-i3.cali'), (1452190937, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t64-s268435456-i3.cali'), (2524911101, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t64-s16777216-i3.cali'), (2544113541, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t64-s65536-i3.cali'), (2883885915, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t64-s4194304-i3.cali'), (3231185819, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t64-s67108864-i3.cali'), (4086191543, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t64-s1048576-i3.cali')])
profile
486381914        262144
1452190937    268435456
2524911101     16777216
2544113541        65536
2883885915      4194304
3231185819     67108864
4086191543      1048576
Name: InputSize, dtype: int64
1
OrderedDict([(736894305, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t128-s268435456-i3.cali'), (1096025866, '/scratch/user/anjalik02/Project-Bitonic/CUDA/t128-s4194304-i3.cali'), (2465771436, 